# 課題の回答を回収します
---

課題の回答を記述したnotebookなどの回答ファイルを回収します。
* 回収したファイルは、`.tgz` ファイルにアーカイブして、指定されたフォルダに格納します。
* notebookファイルは、HTML ファイルに変換して同梱します。

## 1. 回収対象ユーザのメールアドレスを指定します

`STUDENT_LIST` に、回答を回収する対象ユーザのメールアドレスを列挙します。
```
STUDENT_LIST = '''
student-a01@example.com
student-a02@example.com
'''
```

** 次のセルを実行して、回答を回収する対象ユーザのメールアドレスを設定してください。**

In [ ]:
# 以下に、回答を回収する対象ユーザのメールアドレスを列挙してください。
STUDENT_LIST = '''#ここから#
# GROUP A
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''#ここまで#

## 2. 回収するファイルおよびフォルダを指定します

`ANSWER_LIST` に、回収するファイルおよびフォルダを、改行区切りで列挙して下さい。
```
ANSWER_LIST = '''
answer1.ipynb
answer2.ipynb
answers-folder/
user-folder/answer-data.txt
'''
```

** 次のセルで回収対象のファイル/フォルダを設定して、セルを実行して下さい。**

In [ ]:
ANSWER_LIST = '''
./StudentCreated01.ipynb
StudentCreatedFolder//StudentCreated02.ipynb
StudentCreatedFolder/
./././TestTextbook-0625.ipynb
NotFound.ipynb
NotFound/NotFound/not-found/notexist.ipynb
'''

## 3. 回収したファイルのアーカイブ先を指定します

回収したファイルをアーカイブする先のフォルダ名/ファイル名を `ARCHIVE_FILE` に設定して下さい。
```
ARCHIVE_FILE='student-answers/0123/answers.tgz'
```
- ホームディレクトリ直下からの相対パスで指定します。
- フォルダを指定した場合には、フォルダ以下に格納されます。
- フォルダが存在しない場合には、自動的に作成されます。

** 次のセルでアーカイブファイル名を指定して、セルを実行して下さい。**

In [ ]:
ARCHIVE_FILE='student-answers/0123/answers.tgz'

## 4. 次のセルを実行して、回答を回収して下さい

* 回答アーカイブファイルは、上記で指定したファイルとして格納されます。

In [ ]:
import os, sys, re, hashlib, string, tempfile, shutil, subprocess

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

def ignore_files(path, names):
    ignore_set=set()
    for name in names:
        if name.startswith('.'):
            ignore_set.add(name)
    return ignore_set
        

#
target_students=set()
for line in STUDENT_LIST.splitlines():
    s = re.sub(r'#.*$', '', line).strip()
    if(0 < len(s)):
        target_students.add(s)

#
target_files=set()
for line in ANSWER_LIST.splitlines():
    s = re.sub(r'#.*$', '', line).strip()
    if(0 < len(s)):
        target_files.add(os.path.normpath(s))

#
archive_file_path = os.path.expanduser('~/{}'.format(os.path.normpath(ARCHIVE_FILE)))
archive_file_dir = os.path.dirname(archive_file_path)
if not os.path.exists(archive_file_dir):
    os.makedirs(archive_file_dir)

#
with tempfile.TemporaryDirectory() as tmp_dir:
    for student in target_students:
        print('STUDENT[{}]の回答を回収します'.format(student))
        user_name = get_username_from_mail_address(student)
        user_dir = '/home/jupyter/workspace/{}'.format(user_name)
        if not os.path.isdir(user_dir):
            print('WARNING: {}: 生徒フォルダが見つかりません'.format(student), file=sys.stderr)
            print('-> 失敗')
            continue
        for file in sorted(target_files, key=len):
            basename = os.path.basename(file)
            dirname = os.path.dirname(file)
            srcdir = user_dir
            dstdir = '{}/{}'.format(tmp_dir, user_name)
            if dirname: # with directory name(s)
                srcdir = '{}/{}'.format(srcdir, dirname)
                dstdir = '{}/{}'.format(dstdir, dirname)
            srcfile = '{}/{}'.format(srcdir, basename)
            dstfile = '{}/{}'.format(dstdir, basename)
            if not os.path.exists(srcfile):
                print('WARNING: {}: {}: 回答が見つかりません'.format(student, file), file=sys.stderr)
                continue
            if os.path.isdir(srcfile):
                shutil.copytree(srcfile, dstfile, ignore=ignore_files)
            else:
                os.makedirs(dstdir, exist_ok=True)
                shutil.copy2(srcfile, dstfile)
        print('-> 完了')
    try:
        tarcmd = 'tar -cz -C "{}" -f "{}" ./*'.format(tmp_dir, archive_file_path)
        cp = subprocess.run(tarcmd, shell=True, cwd=tmp_dir )
        if cp.returncode == 0:
            print('')
            print('回答アーカイブファイルの作成に成功しました：{}'.format(archive_file_path))
        else:
            print('ERROR: tar コマンドが失敗しました', file=sys.stderr)
    except OSError as e:
        print('ERROR: tar コマンドが実行できません：{}'.format(e), file=sys.stderr)